In [36]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 40]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
3022  prop-1       40   25    2    0    8   59   150   0   8  ...  0.416667   
3023  prop-1       40    7    3    0    2   14     9   0   2  ...  1.000000   
3024  prop-1       40    2    1    0    4    2     1   0   4  ...  0.000000   
3025  prop-1       40    1    1    0    0    5     0   0   0  ...  0.000000   
3026  prop-1       40    3    1    0    1    4     1   1   0  ...  1.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
7070  prop-1       40    4    3    1    5   17     2   1   4  ...  0.800000   
7071  prop-1       40    4    1    0    9   20     2   1   8  ...  0.750000   
7072  prop-1       40    2    4    0    7    8     1   1   6  ...  0.000000   
7073  prop-1       40    6    3    0    7   21     5   0   7  ...  1.000000   
7074  prop-1       40    3    1    0    4   11     3   1   3  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
3022    0  0.620690  0.168889   2   14  17.400000      14  1.9600    0  
3023    0  0.933333  0.285714   1    2   5.000000       1  0.5714    0  
3024    0  0.000000  0.666667   0    0   0.000000       1  1.0000    0  
3025    0  0.000000  0.000000   0    0  12.000000       0  0.0000    1  
3026    0  0.000000  0.666667   0    0   3.333333       1  0.6667    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
7070    0  0.866667  0.833333   1    1  23.250000       4  1.5000    0  
7071    1  0.000000  0.444444   0    0  18.000000       3  1.0000    0  
7072    0  0.937500  0.750000   1    1  14.500000       2  1.0000    0  
7073    0  0.886364  0.366667   2    6  20.666667       4  2.0000    0  
7074    0  0.000000  0.333333   0    0  22.333333       3  1.6667    0  

[4053 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc   dam  moa  \
4698    2    1    0   16   17     0  12   4    2  0.000000  115  1.00    0   
4785    6    1    0    2    8     5   2   0    6  0.800000   47  1.00    0   
3990    1    1    0    3    1     0   3   0    1  2.000000    1  0.00    0   
4940    6    1    0    9   69     1   1   9    3  0.650000  671  0.75    0   
3439    2    5    0    7   18     1   0   7    2  2.000000   49  0.00    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...   ...  ...   
4103    7    4    5   40   22    19  38   2    7  0.583333   83  0.50    0   
6228    3    2    0    9   13     1   5   4    2  1.000000   65  0.00    0   
6259    5    2    0   36  136    10   0  36    2  1.000000  577  0.00    0   
3667    9    1    0    7   16    32   0   7    9  0.950000   69  0.80    0   
3969    2    1    0    2    5     1   1   1    2  2.000000   16  0.00    0   

           mfa       cam  ic  cbm         amc  max_cc  avg_cc  
4698  0.000000  0.600000   0    0   55.500000      12  6.0000  
4785  0.000000  0.444444   0    0    6.000000       1  0.8333  
3990  0.000000  1.000000   0    0    0.000000       1  1.0000  
4940  0.000000  0.500000   0    0  110.166667      16  3.6667  
3439  0.990291  1.000000   0    0   23.500000       1  0.5000  
...        ...       ...  ..  ...         ...     ...     ...  
4103  0.772727  0.342857   1    2   10.571429       2  1.2857  
6228  0.866667  0.666667   0    0   20.333333       2  1.0000  
6259  0.500000  0.300000   2    2  113.800000       2  1.0000  
3667  0.500000  1.000000   0    0    6.111111       2  1.1111  
3969  0.000000  0.750000   0    0    7.000000       2  1.0000  

[3242 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
3672    8    1    0   10   17    26   5   5    8  0.928571    63  0.500000   
6296    6    5    0    8   11     9   0   8    2  0.600000    34  1.000000   
6193    3    5    0    4    8     1   1   3    3  1.000000    34  1.000000   
6116    5    1    0    6   19     0   2   4    4  0.000000   107  1.000000   
3167   31    3    0   11   75   247   0  11   27  0.902899   782  0.347826   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
6161    3    4    0    3   37     1   0   3    2  1.125000   228  0.500000   
5713    3    2    1   15   23     3   1  14    1  2.000000   160  0.000000   
4491    2    3    0    5    6     1   0   5    1  2.000000    25  1.000000   
3716   50    2    0   60  212  1049  10  50   44  0.893250  1371  0.923077   
4321    5    2    3    4   16     2   4   0    4  0.750000    81  1.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
3672    0  0.000000  0.275000   0    0   6.625000       1  0.8750  
6296    1  0.871795  0.366667   1    4   4.500000       1  0.8333  
6193    0  0.973684  0.555556   0    0   9.666667       2  1.3333  
6116    0  0.000000  0.450000   0    0  20.000000       7  2.2000  
3167    0  0.654321  0.121588   3   20  23.483871      23  2.4839  
...   ...       ...       ...  ..  ...        ...     ...     ...  
6161    0  0.970588  0.666667   3    5  73.666667       1  0.3333  
5713    0  0.750000  0.571429   1    1  52.333333      10  5.0000  
4491    0  0.928571  0.750000   1    1  11.000000       4  2.0000  
3716    0  0.678571  0.131973   0    0  26.160000       5  1.7200  
4321    0  0.714286  0.433333   1    2  14.600000       2  1.4000  

[811 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

4698    0
4785    0
3990    0
4940    0
3439    0
       ..
4103    0
6228    0
6259    3
3667    0
3969    0
Name: bug, Length: 3242, dtype: int64

In [11]:
y_test

3672    0
6296    0
6193    0
6116    0
3167    0
       ..
6161    0
5713    0
4491    0
3716    0
4321    0
Name: bug, Length: 811, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[  2.        ,   1.        ,   0.        , ...,  55.5       ,
         12.        ,   6.        ],
       [  6.        ,   1.        ,   0.        , ...,   6.        ,
          1.        ,   0.8333    ],
       [  1.        ,   1.        ,   0.        , ...,   0.        ,
          1.        ,   1.        ],
       ...,
       [  5.        ,   2.        ,   0.        , ..., 113.8       ,
          2.        ,   1.        ],
       [  9.        ,   1.        ,   0.        , ...,   6.11111111,
          2.        ,   1.1111    ],
       [  2.        ,   1.        ,   0.        , ...,   7.        ,
          2.        ,   1.        ]])

In [14]:
x_test_np

array([[ 8.        ,  1.        ,  0.        , ...,  6.625     ,
         1.        ,  0.875     ],
       [ 6.        ,  5.        ,  0.        , ...,  4.5       ,
         1.        ,  0.8333    ],
       [ 3.        ,  5.        ,  0.        , ...,  9.66666667,
         2.        ,  1.3333    ],
       ...,
       [ 2.        ,  3.        ,  0.        , ..., 11.        ,
         4.        ,  2.        ],
       [50.        ,  2.        ,  0.        , ..., 26.16      ,
         5.        ,  1.72      ],
       [ 5.        ,  2.        ,  3.        , ..., 14.6       ,
         2.        ,  1.4       ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[  2.        ,   1.        ,   0.        , ...,  55.5       ,
          12.        ,   6.        ]],

       [[  6.        ,   1.        ,   0.        , ...,   6.        ,
           1.        ,   0.8333    ]],

       [[  1.        ,   1.        ,   0.        , ...,   0.        ,
           1.        ,   1.        ]],

       ...,

       [[  5.        ,   2.        ,   0.        , ..., 113.8       ,
           2.        ,   1.        ]],

       [[  9.        ,   1.        ,   0.        , ...,   6.11111111,
           2.        ,   1.1111    ]],

       [[  2.        ,   1.        ,   0.        , ...,   7.        ,
           2.        ,   1.        ]]])

In [18]:
test_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 7, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv40/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
33/33 [==============================] - 5s 11ms/step - loss: 0.4687 - mse: 0.4687 - mae: 0.2509 - root_mean_squared_error: 0.6846 - mean_squared_logarithmic_error: 0.0945
Epoch 2/100
33/33 [==============================] - 0s 6ms/step - loss: 0.4459 - mse: 0.4459 - mae: 0.3107 - root_mean_squared_error: 0.6677 - mean_squared_logarithmic_error: 0.0918
Epoch 3/100
33/33 [==============================] - 0s 6ms/step - loss: 0.4245 - mse: 0.4245 - mae: 0.2678 - root_mean_squared_error: 0.6515 - mean_squared_logarithmic_error: 0.0824
Epoch 4/100
33/33 [==============================] - 0s 6ms/step - loss: 0.4013 - mse: 0.4013 - mae: 0.2386 - root_mean_squared_error: 0.6335 - mean_squared_logarithmic_error: 0.0769
Epoch 5/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3874 - mse: 0.3874 - mae: 0.2335 - root_mean_squared_error: 0.6224 - mean_squared_logarithmic_error: 0.0757
Epoch 6/100
33/33 [==============================] - 0s 6ms/step - loss: 0

33/33 [==============================] - 0s 6ms/step - loss: 0.3155 - mse: 0.3155 - mae: 0.2232 - root_mean_squared_error: 0.5617 - mean_squared_logarithmic_error: 0.0666
Epoch 89/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3123 - mse: 0.3123 - mae: 0.1938 - root_mean_squared_error: 0.5588 - mean_squared_logarithmic_error: 0.0595
Epoch 90/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2939 - mse: 0.2939 - mae: 0.2088 - root_mean_squared_error: 0.5421 - mean_squared_logarithmic_error: 0.0623
Epoch 91/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3029 - mse: 0.3029 - mae: 0.2052 - root_mean_squared_error: 0.5504 - mean_squared_logarithmic_error: 0.0632
Epoch 92/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3036 - mse: 0.3036 - mae: 0.2086 - root_mean_squared_error: 0.5510 - mean_squared_logarithmic_error: 0.0618
Epoch 93/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3006 - mse: 0.300

33/33 [==============================] - 0s 6ms/step - loss: 0.2893 - mse: 0.2893 - mae: 0.1932 - root_mean_squared_error: 0.5379 - mean_squared_logarithmic_error: 0.0622
Epoch 75/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2833 - mse: 0.2833 - mae: 0.1973 - root_mean_squared_error: 0.5322 - mean_squared_logarithmic_error: 0.0613
Epoch 76/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2951 - mse: 0.2951 - mae: 0.1904 - root_mean_squared_error: 0.5432 - mean_squared_logarithmic_error: 0.0612
Epoch 77/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2987 - mse: 0.2987 - mae: 0.1964 - root_mean_squared_error: 0.5465 - mean_squared_logarithmic_error: 0.0600
Epoch 78/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2876 - mse: 0.2876 - mae: 0.1831 - root_mean_squared_error: 0.5363 - mean_squared_logarithmic_error: 0.0577
Epoch 79/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3012 - mse: 0.301

33/33 [==============================] - 0s 6ms/step - loss: 0.2912 - mse: 0.2912 - mae: 0.2090 - root_mean_squared_error: 0.5397 - mean_squared_logarithmic_error: 0.0610
Epoch 61/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2836 - mse: 0.2836 - mae: 0.2004 - root_mean_squared_error: 0.5326 - mean_squared_logarithmic_error: 0.0596
Epoch 62/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2912 - mse: 0.2912 - mae: 0.2043 - root_mean_squared_error: 0.5396 - mean_squared_logarithmic_error: 0.0627
Epoch 63/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2931 - mse: 0.2931 - mae: 0.2014 - root_mean_squared_error: 0.5414 - mean_squared_logarithmic_error: 0.0605
Epoch 64/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2858 - mse: 0.2858 - mae: 0.1929 - root_mean_squared_error: 0.5346 - mean_squared_logarithmic_error: 0.0578
Epoch 65/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2961 - mse: 0.296

33/33 [==============================] - 0s 6ms/step - loss: 0.3132 - mse: 0.3132 - mae: 0.2031 - root_mean_squared_error: 0.5596 - mean_squared_logarithmic_error: 0.0658
Epoch 47/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3118 - mse: 0.3118 - mae: 0.2134 - root_mean_squared_error: 0.5584 - mean_squared_logarithmic_error: 0.0656
Epoch 48/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3089 - mse: 0.3089 - mae: 0.2059 - root_mean_squared_error: 0.5558 - mean_squared_logarithmic_error: 0.0640
Epoch 49/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2979 - mse: 0.2979 - mae: 0.2039 - root_mean_squared_error: 0.5458 - mean_squared_logarithmic_error: 0.0623
Epoch 50/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2949 - mse: 0.2949 - mae: 0.1884 - root_mean_squared_error: 0.5431 - mean_squared_logarithmic_error: 0.0623
Epoch 51/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3218 - mse: 0.321

...

33/33 [==============================] - 0s 6ms/step - loss: 0.3975 - mse: 0.3975 - mae: 0.2355 - root_mean_squared_error: 0.6305 - mean_squared_logarithmic_error: 0.0770
Epoch 5/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3831 - mse: 0.3831 - mae: 0.2241 - root_mean_squared_error: 0.6190 - mean_squared_logarithmic_error: 0.0727
Epoch 6/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3709 - mse: 0.3709 - mae: 0.2203 - root_mean_squared_error: 0.6090 - mean_squared_logarithmic_error: 0.0731
Epoch 7/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3662 - mse: 0.3662 - mae: 0.2137 - root_mean_squared_error: 0.6051 - mean_squared_logarithmic_error: 0.0704
Epoch 8/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3558 - mse: 0.3558 - mae: 0.2151 - root_mean_squared_error: 0.5965 - mean_squared_logarithmic_error: 0.0699
Epoch 9/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3566 - mse: 0.3566 - m

33/33 [==============================] - 0s 6ms/step - loss: 0.2702 - mse: 0.2702 - mae: 0.1882 - root_mean_squared_error: 0.5199 - mean_squared_logarithmic_error: 0.0564
Epoch 92/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2820 - mse: 0.2820 - mae: 0.1933 - root_mean_squared_error: 0.5311 - mean_squared_logarithmic_error: 0.0606
Epoch 93/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2550 - mse: 0.2550 - mae: 0.1864 - root_mean_squared_error: 0.5049 - mean_squared_logarithmic_error: 0.0545
Epoch 94/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2938 - mse: 0.2938 - mae: 0.2022 - root_mean_squared_error: 0.5420 - mean_squared_logarithmic_error: 0.0593
Epoch 95/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2722 - mse: 0.2722 - mae: 0.2049 - root_mean_squared_error: 0.5218 - mean_squared_logarithmic_error: 0.0600
Epoch 96/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2775 - mse: 0.277

33/33 [==============================] - 0s 6ms/step - loss: 0.2971 - mse: 0.2971 - mae: 0.1993 - root_mean_squared_error: 0.5451 - mean_squared_logarithmic_error: 0.0614
Epoch 78/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2821 - mse: 0.2821 - mae: 0.1971 - root_mean_squared_error: 0.5312 - mean_squared_logarithmic_error: 0.0600
Epoch 79/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3025 - mse: 0.3025 - mae: 0.1976 - root_mean_squared_error: 0.5500 - mean_squared_logarithmic_error: 0.0622
Epoch 80/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2814 - mse: 0.2814 - mae: 0.1925 - root_mean_squared_error: 0.5305 - mean_squared_logarithmic_error: 0.0615
Epoch 81/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2796 - mse: 0.2796 - mae: 0.1850 - root_mean_squared_error: 0.5288 - mean_squared_logarithmic_error: 0.0580
Epoch 82/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2857 - mse: 0.285

33/33 [==============================] - 0s 6ms/step - loss: 0.2880 - mse: 0.2880 - mae: 0.2032 - root_mean_squared_error: 0.5366 - mean_squared_logarithmic_error: 0.0629
Epoch 64/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2916 - mse: 0.2916 - mae: 0.1981 - root_mean_squared_error: 0.5400 - mean_squared_logarithmic_error: 0.0618
Epoch 65/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3049 - mse: 0.3049 - mae: 0.2008 - root_mean_squared_error: 0.5522 - mean_squared_logarithmic_error: 0.0629
Epoch 66/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3078 - mse: 0.3078 - mae: 0.1968 - root_mean_squared_error: 0.5548 - mean_squared_logarithmic_error: 0.0623
Epoch 67/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3118 - mse: 0.3118 - mae: 0.2046 - root_mean_squared_error: 0.5584 - mean_squared_logarithmic_error: 0.0625
Epoch 68/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2915 - mse: 0.291

33/33 [==============================] - 0s 6ms/step - loss: 0.3075 - mse: 0.3075 - mae: 0.1984 - root_mean_squared_error: 0.5545 - mean_squared_logarithmic_error: 0.0640
Epoch 50/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3010 - mse: 0.3010 - mae: 0.2034 - root_mean_squared_error: 0.5486 - mean_squared_logarithmic_error: 0.0620
Epoch 51/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2944 - mse: 0.2944 - mae: 0.1921 - root_mean_squared_error: 0.5426 - mean_squared_logarithmic_error: 0.0610
Epoch 52/100
33/33 [==============================] - 0s 6ms/step - loss: 0.3162 - mse: 0.3162 - mae: 0.1924 - root_mean_squared_error: 0.5623 - mean_squared_logarithmic_error: 0.0633
Epoch 53/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2970 - mse: 0.2970 - mae: 0.2039 - root_mean_squared_error: 0.5449 - mean_squared_logarithmic_error: 0.0622
Epoch 54/100
33/33 [==============================] - 0s 6ms/step - loss: 0.2978 - mse: 0.297

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv40/'

In [23]:
total_training_time

[27.659157752990723,
 26.115395307540894,
 26.532159566879272,
 26.68152356147766,
 26.675281286239624,
 26.691996574401855,
 26.643417835235596,
 27.100053548812866,
 26.668237686157227,
 26.60029125213623]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 8.912250479062399


In [25]:
total_training_loss_history

[{'loss': [0.46870526671409607,
   0.4458528161048889,
   0.42445990443229675,
   0.40134456753730774,
   0.38737958669662476,
   0.3693571984767914,
   0.3673386871814728,
   0.35554468631744385,
   0.3541782796382904,
   0.34840384125709534,
   0.3441038131713867,
   0.34266379475593567,
   0.34610626101493835,
   0.3341175615787506,
   0.3358035385608673,
   0.3409331142902374,
   0.33345434069633484,
   0.32778042554855347,
   0.3258288502693176,
   0.33101221919059753,
   0.3299878239631653,
   0.32201865315437317,
   0.31824570894241333,
   0.3289776146411896,
   0.32399359345436096,
   0.31570902466773987,
   0.32063451409339905,
   0.3151347041130066,
   0.3221515119075775,
   0.32137537002563477,
   0.31083551049232483,
   0.3244720697402954,
   0.31030964851379395,
   0.30449262261390686,
   0.31421959400177,
   0.3085125684738159,
   0.3092370629310608,
   0.32815617322921753,
   0.3162647783756256,
   0.3076675236225128,
   0.30593740940093994,
   0.30752116441726685,
   0.

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.09200590352217357
avg mae of training: 0.06329539269208909


In [27]:
total_scores

[[0.4266516864299774,
  0.4266516864299774,
  0.20140622556209564,
  0.6531857848167419,
  0.06702440232038498],
 [0.4271063804626465,
  0.4271063804626465,
  0.20844580233097076,
  0.6535337567329407,
  0.07022399455308914],
 [0.4241423010826111,
  0.4241423010826111,
  0.2244051992893219,
  0.651262104511261,
  0.07475598156452179],
 [0.4414019286632538,
  0.4414019286632538,
  0.20156142115592957,
  0.6643808484077454,
  0.07014860212802887],
 [0.42905867099761963,
  0.42905867099761963,
  0.25950300693511963,
  0.6550257205963135,
  0.08379334956407547],
 [0.46842631697654724,
  0.46842631697654724,
  0.2487161010503769,
  0.6844167709350586,
  0.0820879116654396],
 [0.4230271875858307,
  0.4230271875858307,
  0.20952603220939636,
  0.6504054069519043,
  0.06865312904119492],
 [0.3995782136917114,
  0.3995782136917114,
  0.24481961131095886,
  0.6321219801902771,
  0.07208302617073059],
 [0.42912566661834717,
  0.42912566661834717,
  0.2280626744031906,
  0.6550768613815308,
  0.07

In [1]:
a = [[0.4266516864299774,
  0.4266516864299774,
  0.20140622556209564,
  0.6531857848167419,
  0.06702440232038498],
 [0.4271063804626465,
  0.4271063804626465,
  0.20844580233097076,
  0.6535337567329407,
  0.07022399455308914],
 [0.4241423010826111,
  0.4241423010826111,
  0.2244051992893219,
  0.651262104511261,
  0.07475598156452179],
 [0.4414019286632538,
  0.4414019286632538,
  0.20156142115592957,
  0.6643808484077454,
  0.07014860212802887],
 [0.42905867099761963,
  0.42905867099761963,
  0.25950300693511963,
  0.6550257205963135,
  0.08379334956407547],
 [0.46842631697654724,
  0.46842631697654724,
  0.2487161010503769,
  0.6844167709350586,
  0.0820879116654396],
 [0.4230271875858307,
  0.4230271875858307,
  0.20952603220939636,
  0.6504054069519043,
  0.06865312904119492],
 [0.3995782136917114,
  0.3995782136917114,
  0.24481961131095886,
  0.6321219801902771,
  0.07208302617073059],
 [0.42912566661834717,
  0.42912566661834717,
  0.2280626744031906,
  0.6550768613815308,
  0.07615071535110474],
 [0.46493133902549744,
  0.46493133902549744,
  0.2349146008491516,
  0.6818587183952332,
  0.0824856385588646]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [3]:
test_mae

[0.20140622556209564,
 0.20844580233097076,
 0.2244051992893219,
 0.20156142115592957,
 0.25950300693511963,
 0.2487161010503769,
 0.20952603220939636,
 0.24481961131095886,
 0.2280626744031906,
 0.2349146008491516]

In [4]:
sum(test_mae)/10

0.22613606750965118

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.14444832305113475
avg mae: 0.07537868916988373
avg are: 0.03236369639635086


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.2758934944868088
median testing mse: 0.42808252573013306


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.19017530232667923
median testing mae: 0.22623393684625626


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.09817782789468765


In [35]:
test_ARE

[0.11091931,
 0.10978901,
 0.07793044,
 0.11454994,
 0.08591588,
 0.09180709,
 0.09160159,
 0.07635488,
 0.10749418,
 0.104548566]

In [6]:
sum([0.11091931,
 0.10978901,
 0.07793044,
 0.11454994,
 0.08591588,
 0.09180709,
 0.09160159,
 0.07635488,
 0.10749418,
 0.104548566])/10

0.0970910886